# #30DayMapChallenge
## Day 2: Lines

### Methodology References
1. https://towardsdatascience.com/create-interactive-globe-earthquake-plot-in-python-b0b52b646f27/
2. https://plainenglish.io/blog/plot-satellites-real-time-orbits-with-python-s-matplotlib

### Data Source
1. Satellite orbit: https://celestrak.org/NORAD/elements/
2. Global Relief Model (as 3D basemap): https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/data/ice_surface/grid_registered/netcdf/

### Setup

In [ ]:
# Standard Library Imports
import math as m
import datetime as dt
from datetime import datetime, timedelta
import numpy as np

# Plotting Libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import plotly.graph_objects as go
import plotly.io as pio

# Third-Party Scientific Libraries
from netCDF4 import Dataset

# Satellite Propagation (SGP4)
from sgp4.api import Satrec, jday

In [ ]:
# If cannot import modules and system path Kernal is in a custom Anaconda environment (e.g. C:\Users\User\anaconda3\envs\myenv\python.exe)
#import sys
#!{sys.executable} -m pip install sgp4
#!{sys.executable} -m pip install cartopy

In [ ]:
# Load satellite data
noaa = 'noaa_tle.txt'
goes = 'goes_tle.txt'

# Load global relief model
grd = "ETOPO1_Ice_g_gmt4.grd"

### Plot 2D satellite path maps

In [ ]:
# Load TLEs from a text file
def load_tles(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    satellites = []
    for i in range(0, len(lines), 3): # Loops through the file 3 lines at a time (first line is the satellite name and the others are TLE lines)
        name = lines[i].strip()
        line1 = lines[i+1].strip()
        line2 = lines[i+2].strip()
        sat = Satrec.twoline2rv(line1, line2) # Uses the Satrec class from the sgp4 library to convert the TLE lines into a satellite object
        satellites.append((name, sat))
    return satellites

In [ ]:
# Compute lat/lon ground track for a satellite
def compute_ground_track(sat, duration_minutes=90*10, step_minutes=1):
    now = datetime.utcnow()
    lats, lons = [], []
    for t in range(0, duration_minutes, step_minutes): # Loop over time steps
        dt = now + timedelta(minutes=t)
        # Converts the datetime to Julian date format, which is required by the SGP4 propagator.
        jd, fr = jday(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
        # Uses the SGP4 model to compute the satellite’s position r and velocity v at that time
        e, r, v = sat.sgp4(jd, fr)
        # Convert to Latitude and Longitude
        if e == 0:
            x, y, z = r
            r_norm = np.linalg.norm(r)
            lat = np.degrees(np.arcsin(z / r_norm))
            lon = np.degrees(np.arctan2(y, x))
            lats.append(lat)
            lons.append((lon + 180) % 360 - 180)  # Normalize to [-180, 180]
    # Return Track
    return lats, lons

In [ ]:
# Plot the tracks on a world map
def plot_ground_tracks(tle_file):
    satellites = load_tles(tle_file)

    fig = plt.figure(figsize=(14, 7))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.stock_img()
    ax.coastlines()
    ax.set_global()

    for name, sat in satellites:
        lats, lons = compute_ground_track(sat)
        ax.plot(lons, lats, transform=ccrs.Geodetic(), label=name)

    plt.title('Satellite Path')
    plt.legend(loc='lower left', fontsize='small')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_ground_tracks(noaa)

In [ ]:
plot_ground_tracks(goes)

### Plot 3D earth (based on methodology reference 1)

In [ ]:
# Function loads topographic elevation data from a NetCDF file in GMT format
def load_etopo_gmt(filepath, resolution=1.0):
    data = Dataset(filepath, "r")

    # GMT-style variable names
    lon = data.variables['x'][:]
    lat = data.variables['y'][:]
    topo = data.variables['z'][:]

    # Create meshgrid
    lon_grid, lat_grid = np.meshgrid(lon, lat)

    # Downsample
    skip = max(1, int(resolution / (lon[1] - lon[0])))
    lon_grid = lon_grid[::skip, ::skip]
    lat_grid = lat_grid[::skip, ::skip]
    topo = topo[::skip, ::skip]

    return lon_grid, lat_grid, topo

# Converts geographic coordinates (lat/lon) into 3D Cartesian coordinates on a sphere
# Converts degrees to radians and applies spherical coordinate formulas
def latlon_to_sphere(lon, lat, radius=1):
    lon_rad = np.radians(lon)
    lat_rad = np.radians(lat)
    x = radius * np.cos(lat_rad) * np.cos(lon_rad)
    y = radius * np.cos(lat_rad) * np.sin(lon_rad)
    z = radius * np.sin(lat_rad)
# Returns 3D arrays x, y, z representing the globe’s surface 
    return x, y, z

# Function that builds and displays the 3D globe
def plot_topo_globe(grd_path, resolution=1.0):
    lon, lat, topo = load_etopo_gmt(grd_path, resolution=resolution)
    x, y, z = latlon_to_sphere(lon, lat)

    # Custom topographic color scale
    Ctopo = [
        [0.00, 'rgb(0, 0, 70)'], [0.20, 'rgb(0,90,150)'],
        [0.40, 'rgb(150,180,230)'], [0.50, 'rgb(210,230,250)'],
        [0.50001, 'rgb(0,120,0)'], [0.57, 'rgb(220,180,130)'],
        [0.65, 'rgb(120,100,0)'], [0.75, 'rgb(80,70,0)'],
        [0.90, 'rgb(200,200,200)'], [1.00, 'rgb(255,255,255)']
    ]

    surface = go.Surface(
        x=x, y=y, z=z,
        surfacecolor=topo,
        colorscale=Ctopo,
        cmin=-8000,
        cmax=8000,
        showscale=False,
        lighting=dict(ambient=0.8),
        opacity=1.0
    )

    noaxis = dict(showbackground=False, showgrid=False, showline=False,
                  showticklabels=False, ticks='', title='', zeroline=False)

    layout = go.Layout(
        title='3D Spherical Topography Map',
        autosize=False,
        width=1000,
        height=800,
        scene=dict(
            xaxis=noaxis,
            yaxis=noaxis,
            zaxis=noaxis,
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=1),
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
        ),
        paper_bgcolor='black',
        plot_bgcolor='black',
        template='plotly_dark'
    )

    fig = go.Figure(data=[surface], layout=layout)
    fig.show()

In [ ]:
plot_topo_globe(grd, resolution=1.0)

### Plot satellite orbit paths (based on methodology reference 2)

In [ ]:
# Earth constants
mu = 398600.4418   # km^3/s^2
r  = 6781          # Earth radius / reference radius (km)
D  = 24 * 0.997269 # sidereal day (hours)

In [ ]:
# Main function for reading TLE file and plotting all orbits in 3D
def plot_tle_file(tle_file):
    """Read a TLE file (name + line1 + line2 blocks) and plot all orbits in 3D."""
    with open(tle_file, "r") as f:
        lines = f.readlines()

    triplets = parse_tle_triplets(lines)
    if not triplets:
        print("No valid TLE triplets found.")
        return

    fig = plt.figure()
    ax = plt.axes(projection='3d', computed_zorder=False)

    last_epoch = None
    for name, l1, l2 in triplets:
        # Parse epoch and elements
        epoch = parse_epoch_from_l1(l1)
        elems = orbital_elements_from_tle(l2)
        last_epoch = epoch

        a = elems["a"]
        e = elems["e"]
        b = a * m.sqrt(1 - e**2)
        c = a * e

        # Rotation from perifocal to ECI
        R = rotation_matrix(elems["RAAN"], elems["i"], elems["omega"])

        # Sample the orbital ellipse in perifocal frame and rotate
        x, y, z = [], [], []
        for theta in np.linspace(0, 2*m.pi, 200):
            perifocal = np.array([[a*m.cos(theta)],
                                  [b*m.sin(theta)],
                                  [0.0]])
            focus_shift = np.array([[c], [0.0], [0.0]])
            P = R @ (perifocal - focus_shift)
            x.append(P[0,0]); y.append(P[1,0]); z.append(P[2,0])

        # Label: prefer full name line if present
        label = name.strip() if name.strip() else l2[2:7]
        ax.plot(x, y, z, zorder=5, label=label)

    # Earth wireframe
    u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:15j]
    ax.plot_wireframe(r*np.cos(u)*np.sin(v),
                      r*np.sin(u)*np.sin(v),
                      r*np.cos(v),
                      color="b", alpha=0.5, lw=0.6, zorder=0)

    # Title and axes
    if last_epoch is not None:
        plt.title("Orbits (ECI) as of " + last_epoch.strftime("%Y-%m-%d %H:%M:%S UTC"))
    else:
        plt.title("Orbits (ECI)")

    ax.set_xlabel("X (km)")
    ax.set_ylabel("Y (km)")
    ax.set_zlabel("Z (km)")
    ax.set_aspect('equal', adjustable='box')
    ax.xaxis.set_tick_params(labelsize=8)
    ax.yaxis.set_tick_params(labelsize=8)
    ax.zaxis.set_tick_params(labelsize=8)

    # Legend placement
    if len(triplets) < 5:
        ax.legend()
    else:
        fig.subplots_adjust(right=0.8)
        ax.legend(loc='center left', bbox_to_anchor=(1.07, 0.5), fontsize=8)

    plt.show()

# Helper function 1
def parse_tle_triplets(lines):
    """Parse TLE lines into (name, line1, line2) triplets, skipping blanks. Returns a list of (name, line1, line2) tuples."""
    clean = [ln.rstrip("\n") for ln in lines if ln.strip()]
    triplets = []
    i = 0
    while i + 2 < len(clean):
        name = clean[i].strip()
        l1   = clean[i+1].strip()
        l2   = clean[i+2].strip()
        # Validate typical prefixes
        if not (l1.startswith("1 ") and l2.startswith("2 ")):
            # Try to recover if name line missing: shift by one
            if clean[i].startswith("1 ") and clean[i+1].startswith("2 "):
                triplets.append(("UNKNOWN", clean[i], clean[i+1]))
                i += 2
                continue
            else:
                print(f"Skipping malformed TLE block starting at line {i}:")
                print(f"  {clean[i]}")
                if i+1 < len(clean): print(f"  {clean[i+1]}")
                if i+2 < len(clean): print(f"  {clean[i+2]}")
                i += 1
                continue
        triplets.append((name, l1, l2))
        i += 3
    return triplets

# Helper function 2
def parse_epoch_from_l1(l1):
    """Extracts epoch/timestamp (year, day of year, fractional day) from TLE line 1."""
    # Year (YY) and day-of-year with fractional part (DDD.dddddd)
    yy   = int(l1[18:20])
    dddf = float(l1[20:32])
    # Determine century/convert to full year
    current_two_digit = int(dt.date.today().year % 100)
    year_full = 1900 + yy if yy > current_two_digit else 2000 + yy
    # Convert day-of-year fractional to datetime
    # Start at Jan 1
    base = dt.datetime(year_full, 1, 1)
    # Day-of-year is 1-based, so subtract 1
    whole_days = int(dddf) - 1
    frac_day   = dddf - int(dddf)
    seconds    = int(round(frac_day * 86400))
    return base + dt.timedelta(days=whole_days, seconds=seconds)

# Helper function 3
def orbital_elements_from_tle(l2):
    """Extract orbital parameters (e, i, RAAN, omega, mean motion) from TLE line 2."""
    # TLE fixed-width fields
    # Inclination (deg): cols 9-16
    i_deg   = float(l2[8:16])
    # RAAN (deg): cols 18-25
    raan_deg = float(l2[17:25])
    # Eccentricity (unitless, implied decimal): cols 27-33
    e_str   = l2[26:33].strip()
    e       = float(f"0.{e_str}")
    # Argument of perigee (deg): cols 35-42
    omega_deg = float(l2[34:42])
    # Mean motion (rev/day): cols 53-63
    n_rev_day = float(l2[52:63])

    # Semi-major axis from mean motion using Kepler's third law
    # n (rad/s) = 2*pi * (rev/day) / (86400 s/day)
    n_rad_s = 2 * m.pi * n_rev_day / 86400.0
    a = (mu / (n_rad_s**2))**(1.0/3.0)

    return {
        "e": e,
        "a": a,
        "i": m.radians(i_deg),
        "RAAN": m.radians(raan_deg),
        "omega": m.radians(omega_deg),
        "n_rev_day": n_rev_day
    }

# Helper function 4
def rotation_matrix(RAAN, i, omega):
    """Construct rotation matrix from perifocal coordinates to ECI (Earth Centered Inertial)."""
    R3_RAAN = np.array([
        [ m.cos(RAAN), -m.sin(RAAN), 0],
        [ m.sin(RAAN),  m.cos(RAAN), 0],
        [ 0,            0,           1]
    ])
    R1_i = np.array([
        [1,           0,          0],
        [0,  m.cos(i), -m.sin(i)],
        [0,  m.sin(i),  m.cos(i)]
    ])
    R3_omega = np.array([
        [ m.cos(omega), -m.sin(omega), 0],
        [ m.sin(omega),  m.cos(omega), 0],
        [ 0,             0,            1]
    ])
    return R3_RAAN @ R1_i @ R3_omega

In [ ]:
plot_tle_file(noaa)

In [ ]:
plot_tle_file(goes)

### Full code to plot satellite paths with background

In [ ]:
# -----------------------------
# Globe helpers
# -----------------------------
# Function to load topographic elevation data from a NetCDF file and downsamples it for performance
def load_etopo_gmt(filepath, resolution=1.0):
    data = Dataset(filepath, "r")
    # Extract longitude, latitude and elevation arrays from the NetCDF file
    lon = data.variables['x'][:]
    lat = data.variables['y'][:]
    topo = data.variables['z'][:]

    # Create 2D grids of longitude and latitude for plotting
    lon_grid, lat_grid = np.meshgrid(lon, lat)

    # Downsample
    step_lon = abs(lon[1] - lon[0])
    skip = max(1, int(resolution / step_lon)) if step_lon != 0 else 1
    lon_grid = lon_grid[::skip, ::skip]
    lat_grid = lat_grid[::skip, ::skip]
    topo = topo[::skip, ::skip]

    data.close()
    return lon_grid, lat_grid, topo

# Convert latitude and longitude grids into 3D Cartesian coordinates on a sphere
def latlon_to_sphere(lon, lat, radius=6371.0):
    lon_rad = np.radians(lon)
    lat_rad = np.radians(lat)
    x = radius * np.cos(lat_rad) * np.cos(lon_rad)
    y = radius * np.cos(lat_rad) * np.sin(lon_rad)
    z = radius * np.sin(lat_rad)
    return x, y, z

# -----------------------------
# TLE parsing + orbit geometry
# -----------------------------
MU_EARTH = 398600.4418       # Earth's gravitational parameter in km^3/s^2
EARTH_RADIUS_KM = 6371.0     # Mean radius of Earth in km

# Parse TLE lines into (name, line1, line2) triplets, skipping blanks
def parse_tle_triplets(lines):
    # Clean blank lines
    clean = [ln.rstrip("\n") for ln in lines if ln.strip()]
    # Groups lines in sets of 3 (name, line 1, line 2)
    triplets = []
    i = 0
    while i + 2 < len(clean):
        name = clean[i].strip()
        l1   = clean[i+1].strip()
        l2   = clean[i+2].strip()
        if l1.startswith("1 ") and l2.startswith("2 "):
            triplets.append((name, l1, l2))
            i += 3
        else:
            # Attempt recovery if name missing
            if clean[i].startswith("1 ") and clean[i+1].startswith("2 "):
                triplets.append(("UNKNOWN", clean[i], clean[i+1]))
                i += 2
            else:
                # Skip malformed line and continue
                i += 1
    # Returns a list of valid TLE triplets
    return triplets

# Extracts the epoch/timestamp from TLE line 1
def parse_epoch_from_l1(l1):
    yy = int(l1[18:20])
    dddf = float(l1[20:32])
    current_two_digit = int(dt.date.today().year % 100)
    year_full = 1900 + yy if yy > current_two_digit else 2000 + yy
    base = dt.datetime(year_full, 1, 1)
    whole_days = int(dddf) - 1  # DOY is 1-based
    frac_day = dddf - int(dddf)
    seconds = int(round(frac_day * 86400))
    return base + dt.timedelta(days=whole_days, seconds=seconds)

# Extracts orbital elements (inclination, RAAN, eccentricity, argument of perigee and mean motion)
# Compute semi-major axis
# Utilises Kepler's third law that the square of a planet's orbital period is directly proportional to the cube of the semi-major axis of its orbit
def orbital_elements_from_tle(l2):
    """Extract e, i, RAAN, omega, mean motion (rev/day) and compute semi-major axis."""
    i_deg      = float(l2[8:16])
    raan_deg   = float(l2[17:25])
    e_str      = l2[26:33].strip()
    e          = float(f"0.{e_str}")
    omega_deg  = float(l2[34:42])
    n_rev_day  = float(l2[52:63])

    # n_rad/s = 2*pi * rev/day / 86400
    n_rad_s = 2.0 * m.pi * n_rev_day / 86400.0
    a = (MU_EARTH / (n_rad_s**2))**(1.0/3.0)

    return {
        "e": e,
        "a": a,
        "i": m.radians(i_deg),
        "RAAN": m.radians(raan_deg),
        "omega": m.radians(omega_deg),
        "n_rev_day": n_rev_day
    }

# Builds a rotation matrix to convert from perifocal to ECI coordinates
def rotation_matrix(RAAN, i, omega):
    R3_RAAN = np.array([
        [ m.cos(RAAN), -m.sin(RAAN), 0],
        [ m.sin(RAAN),  m.cos(RAAN), 0],
        [ 0,            0,           1]
    ])
    R1_i = np.array([
        [1,           0,          0],
        [0,  m.cos(i), -m.sin(i)],
        [0,  m.sin(i),  m.cos(i)]
    ])
    R3_omega = np.array([
        [ m.cos(omega), -m.sin(omega), 0],
        [ m.sin(omega),  m.cos(omega), 0],
        [ 0,             0,            1]
    ])
    return R3_RAAN @ R1_i @ R3_omega

# Generate 3D orbit points in ECI coordinates
def orbit_points_eci(a, e, RAAN, i, omega, n_samples=400):
    # Compute semi-minor axis b and focal offset c
    b = a * m.sqrt(1 - e**2)
    c = a * e
    R = rotation_matrix(RAAN, i, omega)

    x, y, z = [], [], []
    for theta in np.linspace(0, 2*m.pi, n_samples):
        perifocal = np.array([[a*m.cos(theta)], [b*m.sin(theta)], [0.0]])
        focus_shift = np.array([[c], [0.0], [0.0]])
        P = R @ (perifocal - focus_shift)  # ECI coordinate (km)
        x.append(P[0,0]); y.append(P[1,0]); z.append(P[2,0])
    return np.array(x), np.array(y), np.array(z)

# -----------------------------
# Combined plot
# -----------------------------
# Plot a 3D topographic globe and overlay orbits from a TLE file.
def plot_globe_with_tle(tle_file, grd_path, resolution=1.0, output_html="orbit.html", custom_title=None):
    # Load and convert topography
    lon_grid, lat_grid, topo = load_etopo_gmt(grd_path, resolution=resolution)
    xg, yg, zg = latlon_to_sphere(lon_grid, lat_grid, radius=EARTH_RADIUS_KM)
    # Define colour scale
    Ctopo = [
        [0.00, 'rgb(0, 0, 70)'], [0.20, 'rgb(0,90,150)'],
        [0.40, 'rgb(150,180,230)'], [0.50, 'rgb(210,230,250)'],
        [0.50001, 'rgb(0,120,0)'], [0.57, 'rgb(220,180,130)'],
        [0.65, 'rgb(120,100,0)'], [0.75, 'rgb(80,70,0)'],
        [0.90, 'rgb(200,200,200)'], [1.00, 'rgb(255,255,255)']
    ]
    # Create globe surface
    surface = go.Surface(
        x=xg, y=yg, z=zg,
        surfacecolor=topo,
        colorscale=Ctopo,
        cmin=-8000,
        cmax=8000,
        showscale=False,
        lighting=dict(ambient=0.8),
        opacity=1.0,
        name="Topography",
        showlegend=False,
        hoverinfo="skip"
    )

    # Parse TLEs and generate orbits
    with open(tle_file, "r") as f:
        lines = f.readlines()
    triplets = parse_tle_triplets(lines)

    colors = [
        'rgb(255,50,50)', 'rgb(255,255,50)', 'rgb(50,255,255)',
        'rgb(255,50,255)', 'rgb(50,255,50)', 'rgb(255,150,50)',
        'rgb(150,255,150)', 'rgb(150,150,255)'
    ]

    traces = [surface]
    last_epoch = None

    for idx, (name, l1, l2) in enumerate(triplets):
        epoch = parse_epoch_from_l1(l1)
        last_epoch = epoch
        elems = orbital_elements_from_tle(l2)

        x, y, z = orbit_points_eci(
            a=elems["a"],
            e=elems["e"],
            RAAN=elems["RAAN"],
            i=elems["i"],
            omega=elems["omega"],
            n_samples=600
        )

        label = name.strip() if name.strip() else l2[2:7]
        traces.append(go.Scatter3d(
            x=x, y=y, z=z,
            mode='lines',
            line=dict(width=3, color=colors[idx % len(colors)]),
            name=label,
            showlegend=True,
            legendgroup=label
        ))

    # Scene layout
    noaxis = dict(showbackground=False, showgrid=False, showline=False,
                  showticklabels=False, ticks='', title='', zeroline=False)

    title_text = custom_title if custom_title else ("3D Spherical Topography with TLE Orbits" + 
                                                    (f" — Epoch: {last_epoch.strftime('%Y-%m-%d %H:%M:%S UTC')}" if last_epoch else ""))

    layout = go.Layout(
        title=title_text,
        autosize=False,
        width=1200,
        height=900,
        scene=dict(
            xaxis=noaxis,
            yaxis=noaxis,
            zaxis=noaxis,
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=1),
            camera=dict(eye=dict(x=1.6, y=1.6, z=1.2)),
            bgcolor='rgb(10, 10, 30)'
        ),
        paper_bgcolor='rgb(10, 10, 30)',
        template='plotly_dark',
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.98,
            xanchor="right",
            x=0.90,
            bgcolor="rgba(20,20,40,0.9)",
            bordercolor="rgba(255,255,255,0.5)",
            borderwidth=1,
            font=dict(size=12, color="white"),
            orientation='v',
            itemclick='toggle',
            itemdoubleclick='toggleothers'
        ),
        margin=dict(l=0, r=0, t=60, b=0)
    )

    # Render and save
    fig = go.Figure(data=traces, layout=layout)
    fig.show()
    fig.write_html(output_html, include_plotlyjs="cdn")

In [ ]:
# NOAA
plot_globe_with_tle(noaa, grd, output_html="noaa_orbit.html", custom_title="Polar-Orbiting NOAA Satellites")

In [ ]:
# GOES
plot_globe_with_tle(goes, grd, output_html="goes_orbit.html", custom_title="GOES Geostationary Satellite Orbits")